#### Transform orders data - string to json

In [0]:
%sql
SELECT * FROM 
gizmobox.bronze.v_orders;

##### 1. Pre-process json string to fix data quality issues

In [0]:
%sql
SELECT value,
regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date":"\$1"') as fixed_value
FROM gizmobox.bronze.v_orders;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW tv_orders_fixed
AS
SELECT value,
regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date":"\$1"') as fixed_value
FROM gizmobox.bronze.v_orders;

##### 2. Transform json string to json object

In [0]:
%sql
SELECT value, 
      schema_of_json(fixed_value) AS fixed_value
    FROM tv_orders_fixed;

In [0]:
%sql
SELECT from_json(fixed_value, 
                  'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value,
                  fixed_value
        FROM tv_orders_fixed;

##### 3. write data to silver table

In [0]:
%sql
CREATE OR REPLACE TABLE gizmobox.silver.orders_json
AS
SELECT from_json(fixed_value, 
                  'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value
        FROM tv_orders_fixed;

In [0]:
%sql
SELECT * FROM gizmobox.silver.orders_json;